# Метрики качества

Вспомним, что задачи машинного обучения сводятся к следующим:

* Задача регрессии: $Y ⊆ ℝ$
  
$$Q(w; X^l) = ∑_{i=1}^l (a(w, x_i) - y_i) ^2 → \min_w$$


* Задача классификации с 2-я классами: $Y = \{1, -1\}$  
  
$$Q(w; X^l) = ∑_{i=1}^l L(a(w, x_i)y_i) → \min_w$$

где
* $X^l =  (x_i, y_i)_{i=1}^l$ - обучающая выборка,

* $x_i \in ℝ^n$ - объекты

* $y_i$ - ответы

* $a(w, x_i)$ - модель машинного обучения

* $L$ - невозрастающая функция остутпа

### Метрики качества для задач классификации




Confusion Matrix (Матрица ошибок): Фундамент.

Мы можем посмотреть непосредственно саму матрицу ошибок, из которой и считаются дальнейшие метрики: методы `confusion_matrix(y_true, y_pred)`
возвращает матрицу, где главная диагональ — правильные ответы, остальные — ошибки, которую мы потом отображаем с помощью `ConfusionMatrixDisplay(cm, display_labels=...)`

Но более конвенцинальные метрики классификации это:

1. **Accuracy** (Точность) — доля правильных ответов. Понимаем, что эта метрика не подходит при несбалансированных классах.

2. **Precision** (Точность) и **Recall** (Полнота):

* Precision = TP / (TP + FP) -> "Насколько мы аккуратны в своих положительных прогнозах?" (Качество)

* Recall = TP / (TP + FN) -> "Какую долю реальных положительных примеров мы нашли?" (Широта охвата)

3. **F1-Score** (F-мера): Гармоническое среднее Precision и Recall. Баланс между двумя метриками.

4. **ROC-AUC**: ROC-кривая показывает вероятность того, что случайно выбранный положительный пример будет ранжирован выше случайно выбранного отрицательного.

Пример: Поиск мошеннических транзакций (важен Recall) vs. рекомендация новостей в ленту (важен Precision).

### Практика

На прошлых семинарах мы уже работали с [данным датасетом](https://huggingface.co/datasets/Davlan/sib200/viewer/rus_Cyrl). Тогда мы считали только f1 для оценки классифкации текстов на категории. Оценим результат того же пайплайна разными способами:

In [ ]:
!pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.35.3
    Uninstalling huggingface-hub-0.35.3:
      Successfully uninstalled huggingface-hub-0.35.3
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: 

In [ ]:
!pip install transformers -qq

In [ ]:
!pip install evaluate -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
from typing import List, Tuple

In [ ]:
from datasets import load_dataset, Dataset
import evaluate
import numpy as np
from sklearn.metrics import classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import pipeline

Pipeline дообучения модели

In [ ]:
SEED = 42

Загрузим токенайзер

In [ ]:
MODEL_NAME = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
seq = 'Привет о дивный новый мир!'
print(tokenizer.encode(seq))
print(tokenizer.convert_ids_to_tokens(tokenizer.encode(seq)))

[101, 77527, 612, 32136, 1916, 10303, 6913, 106, 102]
['[CLS]', 'Привет', 'о', 'див', '##ный', 'новый', 'мир', '!', '[SEP]']


Загрузим данные

In [ ]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

README.md: 0.00B [00:00, ?B/s]

train.tsv: 0.00B [00:00, ?B/s]

dev.tsv: 0.00B [00:00, ?B/s]

test.tsv: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_datasets = load_dataset(DATASET_NAME, DATASET_LANGUAGE)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 701
    })
    validation: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 99
    })
    test: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 204
    })
})

In [ ]:
list_of_categories = sorted(list(
            set(raw_datasets["train"]["category"]) | set(raw_datasets["validation"]["category"]) |
            set(raw_datasets["test"]["category"])
        ))
indices_of_categories = list(range(len(list_of_categories)))

Токенизируем тексты

In [ ]:
MINIBATCH_SIZE = 64
tokenized_train_set = train_set.map(
    lambda it: tokenizer(it['text'], truncation=True),
    batched=True, batch_size=MINIBATCH_SIZE
)
tokenized_validation_set = validation_set.map(
    lambda it: tokenizer(it['text'], truncation=True),
    batched=True, batch_size=MINIBATCH_SIZE
)

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Создадим вспомогательный объект для трансформации данных в тензор

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Конвертируем категории в целевые метки

In [ ]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
indices_of_categories = list(range(len(list_of_categories)))
n_categories = len(list_of_categories)
print(f'Categories for classification are: {list_of_categories}')
id2label = dict(zip(indices_of_categories, list_of_categories))
label2id = dict(zip(list_of_categories, indices_of_categories))

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [ ]:
labeled_train_set = tokenized_train_set.add_column(
    'label',
    [label2id[val] for val in tokenized_train_set['category']]
)
labeled_validation_set = tokenized_validation_set.add_column(
    'label',
    [label2id[val] for val in tokenized_validation_set['category']]
)

Загружаем наш BERT

In [ ]:
classifier = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=n_categories, id2label=id2label, label2id=label2id
)
# добавьте к строчке выше .cuda , чтобы отправить модель на видеокарту
for param in classifier.parameters(): param.data = param.data.contiguous()

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Пишем способ оценки качества обученной модели

In [ ]:
# Дополним метрики для классификации
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
import numpy as np

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    f1_metric = evaluate.load('f1')
    f1_score = f1_metric.compute(predictions=predictions, references=labels, average='macro')['f1']
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='macro', zero_division=0
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'f1_macro': f1_score
    }

In [ ]:
cls_metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return cls_metric.compute(predictions=predictions, references=labels, average='macro')

Обучаем:

In [ ]:
training_args = TrainingArguments(
    output_dir='rubert_sib200',
    learning_rate=2e-5,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    num_train_epochs=20,
    weight_decay=1e-3,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    logging_steps = 5,
    report_to="none",
    data_seed=SEED,
)

In [ ]:
trainer = Trainer(
    model=classifier,
    args=training_args,
    train_dataset=labeled_train_set,
    eval_dataset=labeled_validation_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-1253556369.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,1.816400,1.696731,0.239728
2,1.546700,1.421574,0.393098
3,1.294600,1.123494,0.613492
4,0.976400,0.833369,0.737852
5,0.558100,0.607963,0.832895
6,0.385900,0.505226,0.848001
7,0.225400,0.459560,0.847098
8,0.145300,0.446317,0.818917
9,0.108500,0.476216,0.809019
10,0.055500,0.513006,0.820295


TrainOutput(global_step=220, training_loss=0.37733876285227863, metrics={'train_runtime': 1125.3411, 'train_samples_per_second': 12.458, 'train_steps_per_second': 0.195, 'total_flos': 422725386983400.0, 'train_loss': 0.37733876285227863, 'epoch': 20.0})

In [ ]:
eval_results = trainer.evaluate()
print("Метрики классификации:")
for metric, value in eval_results.items():
    if 'eval_' in metric:
        print(f"{metric}: {value:.4f}")

Метрики классификации:
eval_loss: 0.4463
eval_f1: 0.8189
eval_runtime: 0.3893
eval_samples_per_second: 254.2730
eval_steps_per_second: 5.1370


###  Оценим качество на отложенной выборке

In [ ]:
classifiсation_pipeline = pipeline('text-classification', model=classifier, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [ ]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(list(test_set['text']))))
y_true = test_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       0.75      0.63      0.69        19
         geography       0.89      1.00      0.94        17
            health       0.95      0.91      0.93        22
          politics       0.97      0.93      0.95        30
science/technology       0.91      0.94      0.92        51
            sports       0.88      0.92      0.90        25
            travel       0.90      0.90      0.90        40

          accuracy                           0.90       204
         macro avg       0.89      0.89      0.89       204
      weighted avg       0.90      0.90      0.90       204



## Метрики качества для задач регрессии

1. **MSE** (Mean Squared Error) / RMSE: буквально среднеквадратичная ошибка. Понимаем, что будет сильно штрафовать за большие ошибки. Всегда ли это нам подходит?

2. **MAE** (Mean Absolute Error): Добавляет интерпретируемость — средняя величина ошибки в единицах целевой переменной.

3. **R²** (Коэффициент детерминации): "Какую долю дисперсии целевой переменной объясняет наша модель?". Шкала от -∞ до 1 (1 — идеальная модель).

В базовом виде метрики есть в `sklearn.metrics`

**`mean_squared_error(y_true, y_pred)`** — *среднеквадратичная ошибка (MSE)*  
- Показывает, насколько сильно предсказания отличаются от реальных значений.  
- Чем меньше MSE, тем лучше.  

**`mean_absolute_error(y_true, y_pred)`** — *средняя абсолютная ошибка (MAE)*  
- Среднее значение абсолютных отклонений предсказаний от реальных значений.  
- Менее чувствительна к выбросам, чем MSE.  

**`r2_score(y_true, y_pred)`** — *коэффициент детерминации (**R²**)*  
- Показывает, насколько хорошо модель объясняет вариацию данных.  
- Значение 1 — идеальное соответствие, 0 — модель не лучше среднего.

In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

In [ ]:
# Создадим синтетические данные для задачи регрессии с помощью make_regression
X, y = make_regression(
    n_samples=1000,
    n_features=10,
    noise=0.1,
    random_state=SEED
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

regressor = RandomForestRegressor(n_estimators=100, random_state=SEED)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

regression_results = {
    'mse': mse,
    'mae': mae,
    'r2_score': r2,
    'rmse': rmse
}

print(regression_results)


{'mse': 2621.793155098221, 'mae': 40.177080494230346, 'r2_score': 0.8445988311470158, 'rmse': np.float64(51.20344866411072)}


* MSE - средний квадрат ошибок (чем меньше, тем лучше)
* RMSE - корень из MSE в исходных единицах
* MAE - средняя абсолютная ошибка
* R² - доля объясненной дисперсии (1 - идеально)

## Метрики для оценки генерации

Проблема: Как оценить качество сгенерированного текста? Классические метрики не подходят.

### N-gram based метрики

1. **BLEU** (для машинного перевода и текстового суммирования): Сравнение с референсными текстами на основе точности n-грамм. Акцент на точность.

2. **ROUGE** (для суммаризации): Сравнение с референсными текстами на основе полноты n-грамм. Акцент на полноту.


### Embedding-based метрики

1. **Косинусное сходство** эмбеддингов: Сравниваем семантические векторные представления гипотезы и референса. Улавливает смысл лучше, чем n-граммы.

2. **BERTScore**: Использует контекстуализированные эмбеддинги моделей типа BERT для попарного сравнения токенов в кандидате и референсе. Более современный и надежный подход.

Используем снова Sib200, чтобы не загружать лишние датасеты. Попробуем суммаризировать отрывки из него. Предположим, что суммаризированные отрывки будут семантически близки описанию категории текста. Проверим это:

"Я увидел кошку"
1. Униграммы - ]Я, увидел, кошку]
2. Биграммы - Я увидел, увидел кошку

"Я увидел кошку"
1. Биграммы - Я  у ув ви

In [ ]:
sample_texts = test_set['text'][:20]

In [ ]:
!pip install sentencepiece -q

In [ ]:
!pip install nltk -q

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from datasets import DatasetDict
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Подготовка синтетических данных для суммаризации

def create_synthetic_summarization_data(texts, categories):
    """Создание синтетических данных для суммаризации на основе категорий"""
    summaries = []

    category_to_summary = {
        'entertainment': "Статья о развлекательных событиях и культурной жизни.",
        'geography': "Географическое описание и информация о местоположении.",
        'health': "Материал о здоровье, медицине и благополучии.",
        'politics': "Политическая новость о государственных делах.",
        'science/technology': "Научно-техническая статья об исследованиях и разработках.",
        'sports': "Спортивная новость о событиях и достижениях.",
        'travel': "Путеводитель и советы для путешественников."
    }

    for category in categories:
        summary = category_to_summary.get(category, "Обзорная статья на различные темы.")
        summaries.append(summary)

    return summaries

train_summaries = create_synthetic_summarization_data(
    train_set['text'], train_set['category']
)
validation_summaries = create_synthetic_summarization_data(
    validation_set['text'], validation_set['category']
)
test_summaries = create_synthetic_summarization_data(
    test_set['text'], test_set['category']
)

In [ ]:
summarization_dataset = DatasetDict({
    'train': Dataset.from_dict({
        'text': train_set['text'],
        'summary': train_summaries
    }),
    'validation': Dataset.from_dict({
        'text': validation_set['text'],
        'summary': validation_summaries
    }),
    'test': Dataset.from_dict({
        'text': test_set['text'],
        'summary': test_summaries
    })
})

print("Датасет для суммаризации:")
print(summarization_dataset)

Датасет для суммаризации:
DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 701
    })
    validation: Dataset({
        features: ['text', 'summary'],
        num_rows: 99
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 204
    })
})


In [ ]:
def preprocess_function_summarization(examples):
    """Токенизация данных для задачи суммаризации"""

    model_inputs = tokenizer(
        examples["text"],
        max_length=64,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"],
            max_length=64,
            truncation=True,
            padding=False
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = summarization_dataset.map(
    preprocess_function_summarization,
    batched=True,
    remove_columns=summarization_dataset["train"].column_names
)

print("Токенизированный датасет для суммаризации:")
print(tokenized_datasets["train"])

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

Токенизированный датасет для суммаризации:
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 701
})


In [ ]:
print(f"Токены текста: {tokenized_datasets["train"]['input_ids'][0]}...")
print(f"Токены суммария: {tokenized_datasets["train"]['labels'][0]}...")

Токены текста: [18258, 9707, 388, 14605, 2122, 605, 11084, 36555, 5172, 15921, 7717, 267, 310, 110800, 324, 661, 3611, 909, 1011, 22666, 261, 310, 73387, 324, 661, 5138, 7279, 2587, 259, 279, 310, 112521, 324, 661, 37870, 279, 27677, 2587, 260, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
Токены суммария: [259, 172986, 48728, 259, 57259, 259, 279, 22145, 259, 411, 11121, 205907, 279, 260, 1]...


In [ ]:
# Декодируйте обратно в текст
input_text = tokenizer.decode(tokenized_datasets["train"]['input_ids'][0])
label_text = tokenizer.decode(tokenized_datasets["train"]['labels'][0])
print(f"Вход: {input_text}")
print(f"Цель: {label_text}")

Вход: Турция с трёх сторон окружена морями: на западе — Эгейским, на севере — Чёрным и на юге — Средиземным.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Цель: Географическое описание и информация о местоположении.</s>


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4ebbc35df420cfc00a41467feae9854f41231f363f4502c12e765705af1d7e70
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
rouge_score = evaluate.load("rouge")

In [ ]:
from rouge_score import rouge_scorer
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    def safe_decode_batch(tokenizer, batch):
        texts = []
        for seq in batch:
            print("seq is: ", seq)
            if hasattr(seq, 'tolist'):
                seq = seq.tolist()

            valid_tokens = [int(t) for t in seq if 0 <= int(t) < tokenizer.vocab_size]
            texts.append(tokenizer.decode(valid_tokens, skip_special_tokens=True))
        return texts

    predictions = np.argmax(predictions, axis=-1)

    decoded_preds = safe_decode_batch(tokenizer, predictions)
    labels_processed = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = safe_decode_batch(tokenizer, labels_processed)

    print("\nПримеры предсказаний и меток:")
    for i in range(min(3, len(decoded_preds))):
        print(f"Pred {i+1}: {decoded_preds[i]}")
        print(f"Label {i+1}: {decoded_labels[i]}")
        print("-" * 50)

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    scores = []
    for pred, ref in zip(decoded_preds, decoded_labels):
        score = scorer.score(ref, pred)
        scores.append(score)

    rouge1_f1 = np.mean([score['rouge1'].fmeasure for score in scores])
    rouge2_f1 = np.mean([score['rouge2'].fmeasure for score in scores])
    rougeL_f1 = np.mean([score['rougeL'].fmeasure for score in scores])

    return {
        'rouge1': round(rouge1_f1 * 100, 4),
        'rouge2': round(rouge2_f1 * 100, 4),
        'rougeL': round(rougeL_f1 * 100, 4),
    }

In [ ]:
batch_size = 8
num_train_epochs = 5
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-summarisation",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    logging_strategy="epoch",
    push_to_hub=False,
    report_to="none"
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True,
    max_length=64,
    pad_to_multiple_of=8,
    return_tensors="pt"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-176899065.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
print("Начинаем обучение модели для суммаризации...")
trainer.train()

Начинаем обучение модели для суммаризации...


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss
88,18.359500
176,12.485400
264,9.554200
352,7.800700
440,6.659800


TrainOutput(global_step=440, training_loss=10.971932983398437, metrics={'train_runtime': 168.4984, 'train_samples_per_second': 20.801, 'train_steps_per_second': 2.611, 'total_flos': 231658458316800.0, 'train_loss': 10.971932983398437, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


TypeError: 'int' object is not iterable

In [ ]:
from tqdm import tqdm

def generate_summaries(model, tokenizer, texts, max_length=50, min_length=10):
    summaries = []

    for text in tqdm(texts, desc="Generating summaries"):
        try:
            inputs = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                max_length=256,
                padding=True
            )

            inputs = {k: v.to(model.device) for k, v in inputs.items()}

            summary_ids = model.generate(
                inputs["input_ids"],
                max_length=max_length,
                min_length=min_length,
                num_beams=2,
                early_stopping=True,
                no_repeat_ngram_size=2,
                repetition_penalty=1.2,
                length_penalty=0.8,
                do_sample=False,
                temperature=0.7,
                pad_token_id=tokenizer.eos_token_id
            )

            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            summaries.append(summary)

            # Очистка памяти (если на GPU)
            # if torch.cuda.is_available():
            #     torch.cuda.empty_cache()

        except Exception as e:
            print(f"Error: {e}")
            summaries.append("")

    return summaries

reference_summaries = test_summaries[:20]
generated_summaries = generate_summaries(model, tokenizer, sample_texts)

In [ ]:
for i, (text, ref_summary) in enumerate(zip(sample_texts, reference_summaries)):
    print(f"\nПример {i+1}:")
    print(f"Исходный текст: {text[:200]}...")
    print(f"Эталонная суммария: {ref_summary}")
    generated_summary = generate_summaries(model, tokenizer, [text])[0]
    print(f"Сгенерированная суммария: {generated_summary}")

Теперь оценим результаты генерации модели двумя метриками: метрикой, основанной на n-gram-мах и метрикой, основанной на эмбеддингах

In [ ]:
!pip install bert_score

In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

In [ ]:
def evaluate_summarization_quality(generated_summaries, reference_summaries):
    """Полная оценка качества суммаризации"""

    rouge_results = rouge_score.compute(
        predictions=generated_summaries,
        references=reference_summaries,
        use_stemmer=True
    )

    bertscore_results = bertscore.compute(
        predictions=generated_summaries,
        references=reference_summaries,
        lang="ru"
    )

    bertscore_avg = {
        'precision': np.mean(bertscore_results['precision']),
        'recall': np.mean(bertscore_results['recall']),
        'f1': np.mean(bertscore_results['f1'])
    }

    return {
        'rouge': rouge_results,
        'bertscore': bertscore_avg
    }


In [ ]:
evaluation_results = evaluate_summarization_quality(generated_summaries, reference_summaries)

print("Результаты оценки суммаризации:")
print("\nROUGE метрики:")
for key, value in evaluation_results['rouge'].items():
    print(f"{key}: {value:.4f}")

print("\nBERTScore метрики:")
for key, value in evaluation_results['bertscore'].items():
    print(f"{key}: {value:.4f}")

### Метрики, основанные на вероятности




1. Любая метрика, подходящая для классификации: для предсказания следующего токена мы оцениваем распределение вероятностей по всем токенам
2. Аппроксимированная кросс-энтропия: $$L(D) = -\sum_{d \in D} \log p(w_1, ..., w_k)$$
3. [Перплексия](https://habr.com/ru/companies/wunderfund/articles/580230/): $$P(D) = e^{-\frac{1}{|D|}L(D)}$$
То есть, число Эйлера в степени кросс-энтропии - Мера неопределенности, показывает, насколько модель не уверена в вероятности того или иного примера.

> Например у последовательности "Мама мыла раму" будет низкая перплексия, потому что этот пример часто встречается в данных. А у "Кали мыла раму" - высокая. Вполне возможно, этот пример часто встречается в тексте

4. Внешние метрики качества - используем модель в задаче, смотрим на метрику качества в задаче.

> Например, для предсказания поисковых подсказок можно смотреть на то, какой процент людей по ним переходит

**Перплексию** можно использовать:
1. Для псевдоразметки данных
2. Как способ выбрать различные решения:
  > Есть вопрос:  
  > "*В каком году родился А.С. Пушкин?  
  > А. 1917, Б. 1799, В. 1657, Г. 1700.*"  
  
Тогда у варианта  
  > "*В каком году родился А.С. Пушкин?  
  > А. 1917, Б. 1799, В. 1657, Г. 1700. Ответ: Б. 1799*".  
Должна быть наименьшая перплексия

3. Как один из инструментов корпусной лингвистики для изучения закономерностей языка
4. Для промпт-инжениринга, ведь нормативные вопросы всегда будут с меньшей перплексией, чем вопросы с редкими словами, не проставленными знаками препинания, странными последовательностями знаков

## LLM as Judge

Идея: Использовать мощную LLM (например, GPT-4) в качестве "судьи" для оценки выходов других моделей.

 \> Создается промпт-шаблон, который включает в себя: задачу,входные данные (input), ответ кандидата-модели, иногда эталонный ответ (reference) и критерии оценки.

 \> LLM-судья выносит вердикт: числовой балл, оценку по шкале (A/B/C/D) или обоснованное текстовое суждение.

Преимущества:

* Можно оценивать креативность, фактологическую точность, безопасность, следование инструкциям — что угодно.

* Часто коррелирует с человеческой оценкой лучше, чем классические метрики.

Риски и ограничения:

* Предвзятость самой LLM-судьи.

* Высокая стоимость и latency.

* Плохая интерпретируемость.

In [ ]:
!pip install groq -q

In [ ]:
import groq
import os

In [ ]:
# Настройка Groq клиента (нужен API ключ)
# Вы можете получить бесплатный ключ на https://console.groq.com/
# os.environ['GROQ_API_KEY'] = 'your_api_key_here'

class LLMJudge:
    def __init__(self, api_key=None):
        if api_key is None:
            api_key = os.getenv('GROQ_API_KEY')
        self.client = groq.Groq(api_key=api_key)

    def evaluate_summary(self, text, reference_summary, predicted_summary):
        """Оценка качества суммаризации с помощью LLM"""

        prompt = f"""
        Текст для суммаризации:
        {text}

        Эталонная суммария:
        {reference_summary}

        Предложенная суммария:
        {predicted_summary}

        Оцени предложенную суммарию по следующим критериям (от 1 до 5, где 5 - отлично):
        1. Полнота - насколько полно переданы ключевые идеи текста
        2. Точность - насколько точно передана информация без искажений
        3. Связность - насколько суммария логична и легко читается
        4. Сжатие - насколько эффективно сокращен текст

        Ответ в формате JSON:
        {{
            "completeness": score,
            "accuracy": score,
            "coherence": score,
            "compression": score,
            "overall_score": average_score,
            "explanation": "краткое объяснение оценки"
        }}
        """

        try:
            response = self.client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model="openai/gpt-oss-120b",
                temperature=0.1
            )

            return response.choices[0].message.content

        except Exception as e:
            print(f"Ошибка при вызове API: {e}")
            return None


In [ ]:
judge = LLMJudge()

llm_scores = []
for i in range(3):
    eval_result = judge.evaluate_summary(
        test_set['text'][i],
        reference_summaries[i],
        predicted_summaries[i]
    )
    llm_scores.append(eval_result)

print("\nОценки LLM для нескольких примеров:")
for i, score in enumerate(llm_scores):
    print(f"Пример {i+1}: {score}")